# Centrality analysis

In [ ]:
import pandas as pd
import numpy as np
import networkx as nx
import analysis_helper_functions as ahf
from operator import itemgetter
pd.set_option('display.max_columns', None)

## Get centralities

In [ ]:
#G = nx.read_graphml("./level1.graphml")
#analysis = ahf.create_centralities(G)
#analysis.to_csv('level1_analysis.csv', index=False)

## Load data

In [ ]:
analysis = pd.read_csv("../data/follower_analysis.csv")

users = pd.read_csv("../data/users_with_friends.csv")
static_user = pd.read_csv("../data/static_users.csv")
users = pd.concat([users, static_user])
users = users.drop_duplicates(['id']).astype({'id_str': str})

followings = pd.read_csv("../data/followings_with_friends.csv")
followings.drop_duplicates(inplace=True)

hs_ids = list(static_user['id'])

In [ ]:
analysis = analysis.astype({'id': np.int64})
user_cols = ['id', 'followers_count', 'friends_count']
analysis_cols = ['id', 'name', 'degree', 'out_degree', 'in_degree', 'betweenness_centrality', 'closeness_centrality', 'eigenvector_centrality']

user_analysis = analysis[analysis_cols].merge(users[user_cols], on='id', how='left')
user_analysis.drop_duplicates('id', inplace=True)

### User data

* HSD Account ist der jüngste Account (2015) und der Account der TH Köln ist der älteste (2007)
* Die 3 größten Accounts sind verifiziert (RWTH Aachen, TH Köln und FH Münster)

In [ ]:
users[users['id'].isin(hs_ids)]
hochschulen = list(users[users['id'].isin(hs_ids)]['name'])

In [ ]:
follower_count = list(users[users['id'].isin(hs_ids)]['followers_count'])
ahf.create_bar_plot_for_hs(hochschulen, follower_count, "Anzahl der Follower", "Vergleich der Anzahl der Follower der Hochschulen", 0.2)

Im Vergleich zu der Anzahl der Follower hat die FH Münster verhältnismäßig viele Tweets (inkl. Retweets) veröffentlicht

In [ ]:
statuses_count = list(users[users['id'].isin(hs_ids)]['statuses_count'])
ahf.create_bar_plot_for_hs(hochschulen, statuses_count, "Anzahl der Tweets (inkl. Retweets)", "Vergleich der Anzahl der Tweets der Hochschulen", 0.2)

In [ ]:
listed_count = list(users[users['id'].isin(hs_ids)]['listed_count'])
ahf.create_bar_plot_for_hs(hochschulen, listed_count, "Anzahl der öffentlichen Listen", "Vergleich der Anzahl der öffentlichen Listen in denen die einzelnen Hochschulen Mitglieder sind", 0.13)

In [ ]:
follower_count = list(users[users['id'].isin(hs_ids)]['friends_count'])
ahf.create_bar_plot_for_hs(hochschulen, follower_count, "Anzahl der Friends", "Vergleich der Anzahl der Friends der Hochschulen", 0.15)

## Welchen HS folgen die einzelnen HS?

<center>
    <img src="./img/hochschulen_follower_vergleich.jpg" width=800px>
</center>

### Followings der HSD

In [ ]:
ahf.get_followed_hs_by_user(followings, users, 3063800235, hs_ids)

# Degree Centrality

## Out Degree

-> Insgesamt gibt es 32 User mit dem out degree 8 -> Diese User folgen allen betrachteten Hochschulen

**Aufteilung der User in Kategorien**
1. Personen
    - Uwe Kleinkes (Professor for Technology Marketing at Hochschule Hamm-Lippstadt)
    - steffiflei (Erasmus+)
    - Svenja Schulze (Bundesministerin für wirtschaftliche Zusammenarbeit und Entwicklung)
    - Elmar Bigler
    - Brigitte Wende
2. Studieninformationsprotale
    - studieren.de
    - Hochschulmarketing
    - Study In Germany
3. Nicht betrachtete Universitäten, Hochschulen
    - Universität Konstanz
    - Bergische Universität Wuppertal	
    - TH Georg Agricola (nur Research Management)
    - Hochschule Neu-Ulm
4. Produkte/ Dienste für Hochschulen
    - open-access.network
    - UniNow
    - alumni-clubs.net
    - PROvendis GmbH + Susanne Salwiczek
    - Studentischer Akkreditierungspool
    - StudySmarter
5. Presse
    - Hochschulspiegel
    - Daily Research @SurveyCircle 
6. sonstige Unternehmen/ Institute
    - ILS
    - VDI Technologiezentrum GmbH	
    - SPECTRUM AG
    - Zentrum für Wissenschaftsmanagement e.V. Speyer
    - open-access.network
7. NRW Institutionen
    - zdi.NRW
    - Netzwerk Frauen- und Geschlechterforschung NRW
    - EFRE.NRW
    - MKW_NRW
    - GI NRW Life Sciences
    - PK NRW Medien und Interaktion
    - Aufstiegsstipendium (Landesebene)

In [ ]:
out_degree_df = user_analysis.sort_values('out_degree', ascending=False).reset_index(drop=True)[:1000]
out_degree_df = ahf.create_degree_df(out_degree_df, followings)

In [ ]:
out_degree_df.head(50)

45 Nutzer folgen 7 Hochschulen, davon folgen:
* 5 Nutzer nicht der HSD
* 7 Nutzer nicht der RWTH
* 1 Nutzer nicht der TH Köln
* 3 Nutzer nicht der FH Münster
* 1 Nutzer nicht der HS Rhein-Waal
* 8 Nutzer nicht der HS Niederrhein
* 18 Nutzer nicht der FH Dortmund
* 2 Nutzer nicht der HS Bochum

In [ ]:
out_degree_copy = out_degree_df.copy()
ahf.create_follower_friend_cols(out_degree_copy, followings, hs_ids)
out_degree_copy[(out_degree_copy['out_degree'] == 7) & (out_degree_copy['follower_of_hsd'] == False)]

In [ ]:
out_degree_df[out_degree_df['id'].isin(hs_ids)]

## In Degree

Zwei Intutitionen werden von allen Hochschulen gefolgt:
- MKW_NRW 
- Ruhr-Universität Bochum

In [ ]:
in_degree_df = user_analysis.sort_values('in_degree', ascending=False).reset_index(drop=True)[8:20]
in_degree_df = ahf.create_degree_df(in_degree_df, followings)

In [ ]:
in_degree_df

## Degree

In [ ]:
degree_df = user_analysis.sort_values('degree', ascending=False).reset_index(drop=True)[8:15]
degree_df = ahf.create_degree_df(degree_df, followings)

In [ ]:
degree_df

# Closeness centrality

* Schätzt, wie schnell der Informationsfluss durch einen bestimmten Knoten zu anderen Knoten ist.

- Welche Punkte sind am dichtesten an allen anderen Punkten dran

-> Welches sind die zentralsten Nutzer?



In [ ]:
closeness_sorted = user_analysis.sort_values('closeness_centrality', ascending=False).reset_index(drop=True)
closeness_sorted.head(10)

### Die Hochschulen im Vergleich:

In [ ]:
hs_closeness_sorted = closeness_sorted[closeness_sorted['id'].isin(hs_ids)]
hs_closeness_sorted = hs_closeness_sorted[['id', 'closeness_centrality', 'name', 'followers_count', 'friends_count']]
ahf.create_additional_info_cols(hs_closeness_sorted, followings, hs_ids)

In [ ]:
hs_closeness_sorted

### Welche User sind am dichtesten an allen anderen Usern dran?

In [ ]:
extract_closeness_sorted = closeness_sorted.iloc[:1000].copy()
extract_closeness_sorted = extract_closeness_sorted[['id', 'closeness_centrality', 'name', 'followers_count', 'friends_count']]
ahf.create_additional_info_cols(extract_closeness_sorted, followings, hs_ids)

In [ ]:
extract_closeness_sorted.head()

* Die Closeness Centrality wird durch die RWTH Aachen dominiert. Alle User, denen die RWTH folgt haben eine hohe Closeness Centrality, vermutlich aufgrund der hohen Follower Anzahl der RWTH. 
* Die anderen Hochschulen befinden sich dazwischen. Bis zum Platz 425 folgt die RWTH allen Usern, außer den einzelnen Hochschulen.
* Friends der RWTH (417) + Anzahl der betrachteten Hochschulen (8) = 425

In [ ]:
extract_closeness_sorted_copy = extract_closeness_sorted.copy()
ahf.create_follower_friend_cols(extract_closeness_sorted_copy, followings, hs_ids)

In [ ]:
extract_closeness_sorted_copy.iloc[:426][extract_closeness_sorted_copy['friend_of_rwth'] == False]

Die HSD ist im Ranking der Hochschulen auf Platz 5 und somit vor anderen Hochschulen mit mehr Followern.
Vermutung: Die HSD folgt vielen Usern, denen die RWTH auch folgt. Diese haben alle eine hohe Centrality.
Allerdings:
* HS Bochum ist auf Platz 4 und folgt keinem Usern, denen die RWTH Aachen auch folgt.
* HS Rhein-Waal ist auf Platz 6 und folgt mehr Usern, denen die RWTH Aachen folgt.

Eventuell Sortierungskriterien:
1. Followeranzahl
2. Anzahl der Hochschulen die folgen
3. Anzahl der Hochschulen die gefolgt werden

-> Anzahl der Hochschulen als Follower passt, bis auf RWTH Aachen und TH Köln. Diese fallen aus dem Raster, haben allerdings auch sehr viel mehr Follower als die anderen Hochschulen

In [ ]:
values = list(extract_closeness_sorted_copy[extract_closeness_sorted_copy['friend_of_rwth'] == True].loc[:426, ['follower_of_hsd', 'follower_of_rhein_waal', 'follower_of_niederrhein', 'follower_of_bochum', 'follower_of_koeln', 'follower_of_dortmund', 'follower_of_muenster', 'follower_of_rwth']].sum())
ahf.create_bar_plot_for_hs(hochschulen, values, 'Anzahl der Follower', 'Anzahl der Follower der Hochschulen bis Platz 425 der Closeness Centrality', 0.1)

In [ ]:
values = list(extract_closeness_sorted_copy[extract_closeness_sorted_copy['friend_of_rwth'] == True].loc[:426, ['friend_of_hsd', 'friend_of_rhein_waal', 'friend_of_niederrhein', 'friend_of_bochum', 'friend_of_koeln', 'friend_of_dortmund', 'friend_of_muenster', 'friend_of_rwth']].sum())
ahf.create_bar_plot_for_hs(hochschulen, values, 'Anzahl der Friends', 'Anzahl der Friends der Hochschulen bis Platz 425 der Closeness Centrality',0.1)

# Betweenness centrality

* Der Wert misst, wie oft ein Knoten im kürzesten Pfad zwischen zwei Knoten liegt.
* Knoten mit einer hohen Betweeness centrality sind interessant, weil sie auf den Kommunikationspfaden liegen und den Informatiosnfluss kontrollieren können. Würden diese Knoten wegbrechen, würde das Netzwerk dadurch stark beeinflusst werden.

-> Kann genutzt werden, um nur die wichtigsten Knoten des Netzwerkes auszuwählen

Viele Verbindungen die andere nicht haben -> User sind Innovatoren und Vermittler in dem Netzwerk

In [ ]:
betweenness_sorted = user_analysis.sort_values('betweenness_centrality', ascending=False).reset_index(drop=True)
betweenness_sorted.head(10)

### Die Hochschulen im Vergleich:

In [ ]:
hs_betweenness_sorted = betweenness_sorted[betweenness_sorted['id'].isin(hs_ids)]
hs_betweenness_sorted = hs_betweenness_sorted[['id', 'betweenness_centrality', 'name', 'followers_count', 'friends_count']]
ahf.create_additional_info_cols(hs_betweenness_sorted, followings, hs_ids)

In [ ]:
hs_betweenness_sorted

* Die betrachteten Hochschulen befinden sich alle auf den ersten Plätzen des Rankings.
* Die HSD steht auf Platz 3 obwohl sie die wenigsten Follower hat


**HSD vs. FH Münster**
* FH Münster hat ca. 4.500 Follower mehr
* FH Münster folgt 2 Hochschulen mit insgesamt ca. 14.000 Followern
* HSD folt 7 Hochschulen mit insgesamt ca. 47.000 Followern

In [ ]:
print('FH Münster:')
print('Follower (insgesamt) der gefolgten Hochschulen: ' + str(ahf.get_followed_hs_by_user(followings, users, 84606793, hs_ids)['followers_count'].sum()))

print('HSD:')
print('Follower (insgesamt) der gefolgten Hochschulen: ' + str(ahf.get_followed_hs_by_user(followings, users, 3063800235, hs_ids)['followers_count'].sum()))

In [ ]:
print('HS Rhein-Waal:')
print('Follower (insgesamt) der gefolgten Hochschulen: ' + str(ahf.get_followed_hs_by_user(followings, users, 265859722, hs_ids)['followers_count'].sum()))

print('HSD:')
print('Follower (insgesamt) der gefolgten Hochschulen: ' + str(ahf.get_followed_hs_by_user(followings, users, 3063800235, hs_ids)['followers_count'].sum()))

### Welche User sind am wichtigsten?

In [ ]:
extract_betweenness_sorted = betweenness_sorted.iloc[:500].copy()
extract_betweenness_sorted = extract_betweenness_sorted[['id', 'betweenness_centrality', 'name', 'followers_count', 'friends_count']]
ahf.create_additional_info_cols(extract_betweenness_sorted, followings, hs_ids)

extract_betweenness_sorted_copy = extract_betweenness_sorted.copy()
ahf.create_follower_friend_cols(extract_betweenness_sorted_copy, followings, hs_ids)

Die Betweeness Centrality wird nicht so stark von der RWTH Aachen dominiert. Bis zum Platz 425 folgt die RWTH Aachen nur ca. 88 Usern.

Die HSD steht auf Platz 3 obwohl sie die wenigsten Follower hat. 
Mögliche Erklärung:
* Die HSD folgt vielen Usern, die eine hohe Betweenness Centrality haben. Sie folgt nach der RWTH Aachen, der TH Köln und der Hochschule Rhein-Waal am meisten Usern mit einer hohen Betweenness Centrality. Diese vier Hochschulen befinden sich auch auf den ersten vier Plätzen nach der Betweenness Centrality. 
* HSD ist auf Platz 3 und HS Rhein-Waal auf Platz 4, obwohl die HS Rhein-Waal mehr Follower hat und mehr Usern folgt die eine hohe Betweenness Centrality haben. Bedeutender Unterschied: die HSD folgt der TH Köln. Diese steht auf Platz 2 des Rankings nach der Betweenness Centrality. Die HS Rhein-Waal folgt dieser nicht.

In [ ]:
values = list(extract_betweenness_sorted_copy[extract_betweenness_sorted_copy['friend_of_rwth'] == True].loc[:426, ['follower_of_hsd', 'follower_of_rhein_waal', 'follower_of_niederrhein', 'follower_of_bochum', 'follower_of_koeln', 'follower_of_dortmund', 'follower_of_muenster', 'follower_of_rwth']].sum())
ahf.create_bar_plot_for_hs(hochschulen, values, 'Anzahl der Follower', 'Anzahl der Follower der Hochschulen bis Platz 425 der Betweenness Centrality', 0.1)

In [ ]:
values = list(extract_betweenness_sorted_copy[extract_betweenness_sorted_copy['friend_of_rwth'] == True].loc[:426, ['friend_of_hsd', 'friend_of_rhein_waal', 'friend_of_niederrhein', 'friend_of_bochum', 'friend_of_koeln', 'friend_of_dortmund', 'friend_of_muenster', 'friend_of_rwth']].sum())
ahf.create_bar_plot_for_hs(hochschulen, values, 'Anzahl der Friends', 'Anzahl der Friends der Hochschulen bis Platz 425 der Betweenness Centrality', 0.1)

## Überschneidungen der Follower

In [ ]:
hsd_follower = followings[followings['follower_of'] == 3063800235]['id']
rwth_follower = followings[followings['follower_of'] == 928008620]['id']
koeln_follower = followings[followings['follower_of'] == 11053712]['id']
muenster_follower = followings[followings['follower_of'] == 84606793]['id']
bochum_follower = followings[followings['follower_of'] == 124155166]['id']
rhein_waal_follower = followings[followings['follower_of'] == 265859722]['id']
niederrhein_follower = followings[followings['follower_of'] == 2776187059]['id']
dortmund_follower = followings[followings['follower_of'] == 103823788]['id']

In [ ]:
hs_follower = {"HSD": hsd_follower, 
               "RWTH Aachen": rwth_follower,
               "TH Koeln": koeln_follower,
               "FH Muenster": muenster_follower,
               "HS Bochum": bochum_follower,
               "HS Rhein Waal": rhein_waal_follower,
               "HS Niederrhein": niederrhein_follower,
               "FH Dortmund": dortmund_follower
}

In [ ]:
ahf.create_hs_heat_map(hs_follower, "Prozentuale Übereinstimmung der Follower der Hochschulen")

* Vermutung: HSD hat eine hohe Betweeness Centrality, da sie viele Verbindungen hat, die andere nicht haben. Die FH Münster hat zwar viel mehr Follower, aber große Überschneidungen mit anderen Hochschulen, insbesondere RWTH Aachen und TH Köln.

-> Prozentual gesehen hat die HSD sehr viele Überschneidungen mit anderen Hochschulen. Sie hat lediglich ca. 19% Follower, die keiner anderen Hochschule folgen. Alle anderen Hochschulen stehen besser da, außer die HS Bochum.

**Anzahl der gemeinsamen Follower mit anderen Hochschulen (Prozentualer Anteil der gemeinsamen Follower)**

In [ ]:
follower_counts = [1743, 21866, 12431, 5256, 1859, 2282, 1864, 1898]
follower_overlap_percentages = ahf.get_overlap_percentages(follower_counts, hs_follower)

In [ ]:
ahf.create_bar_plot_for_hs(hs_follower.keys(), follower_overlap_percentages, 'Anzahl der gemeinsamen Follower (Prozent)', 'Anzahl der gemeinsamen Follower mit anderen Hochschulen', 0.1)

## Überschneidung der Followings

In [ ]:
hsd_followings = followings[followings['id'] == 3063800235]['follower_of']
rwth_followings = followings[followings['id'] == 928008620]['follower_of']
koeln_followings = followings[followings['id'] == 11053712]['follower_of']
muenster_followings = followings[followings['id'] == 84606793]['follower_of']
bochum_followings = followings[followings['id'] == 124155166]['follower_of']
rhein_waal_followings = followings[followings['id'] == 265859722]['follower_of']
niederrhein_followings = followings[followings['id'] == 2776187059]['follower_of']
dortmund_followings = followings[followings['id'] == 103823788]['follower_of']

In [ ]:
hs_followings = {"HSD": hsd_followings, 
               "RWTH Aachen": rwth_followings,
               "TH Koeln": koeln_followings,
               "FH Muenster": muenster_followings,
               "HS Bochum": bochum_followings,
               "HS Rhein Waal": rhein_waal_followings,
               "HS Niederrhein": niederrhein_followings,
               "FH Dortmund": dortmund_followings
}

In [ ]:
ahf.create_hs_heat_map(hs_followings, "Prozentuale Übereinstimmung der Followings der Hochschulen")

**Anzahl der gemeinsamen Followings mit anderen Hochschulen (Prozentualer Anteil der gemeinsamen Followings)**

In [ ]:
followings_counts = [704, 417, 564, 269, 356, 543, 233, 70]
followings_overlap_percentages = ahf.get_overlap_percentages(followings_counts, hs_followings)

In [ ]:
ahf.create_bar_plot_for_hs(hs_followings.keys(), followings_overlap_percentages, 'Anzahl der gemeinsamen Followings (Prozent)', 'Anzahl der gemeinsamen Followings mit anderen Hochschulen', 0.1)

## Eigenvector centrality

* Eine Person mit wenigen Verbindungen kann eine hohe Eigenvector Centrality haben, wenn diese Verbindungen sie mit sehr gut vernetzten anderen Personen verbindet.
* Filtert Personen raus, die nicht nur gut verbunden sind, sondern auch mit anderen verbunden sind, die zentral liegen.

- Finden der einflussreichsten Knoten (Weitergabe von Informationen)

In [ ]:
eigenvector_sorted = user_analysis.sort_values('eigenvector_centrality', ascending=False).reset_index(drop=True)
eigenvector_sorted.head()

### Die Hochschulen im Vergleich

In [ ]:
hs_eigenvector_sorted = eigenvector_sorted[eigenvector_sorted['id'].isin(hs_ids)]
hs_eigenvector_sorted = hs_eigenvector_sorted[['id', 'eigenvector_centrality', 'name', 'followers_count', 'friends_count']]
ahf.create_additional_info_cols(hs_eigenvector_sorted, followings, hs_ids)

In [ ]:
hs_eigenvector_sorted

-> HSD steht im Vergleich zu anderen ganz gut da. Heißt nicht, dass der Account erfolgreicher ist, sondern nur, dass der Informationsfluss erfolgreicher sein könnte als bei anderen 

In [ ]:
extract_eigenvector_sorted = eigenvector_sorted.iloc[:500].copy()
extract_eigenvector_sorted = extract_eigenvector_sorted[['id', 'betweenness_centrality', 'name', 'followers_count', 'friends_count']]
ahf.create_additional_info_cols(extract_eigenvector_sorted, followings, hs_ids)

extract_eigenvector_sorted_copy = extract_eigenvector_sorted.copy()
ahf.create_follower_friend_cols(extract_eigenvector_sorted_copy, followings, hs_ids)

In [ ]:
values = list(extract_eigenvector_sorted_copy[extract_eigenvector_sorted_copy['friend_of_rwth'] == True].loc[:426, ['follower_of_hsd', 'follower_of_rhein_waal', 'follower_of_niederrhein', 'follower_of_bochum', 'follower_of_koeln', 'follower_of_dortmund', 'follower_of_muenster', 'follower_of_rwth']].sum())
ahf.create_bar_plot_for_hs(hochschulen, values, 'Anzahl der Follower', 'Anzahl der Follower der Hochschulen bis Platz 425 der Eigenvector Centrality', 0.1)

In [ ]:
values = list(extract_eigenvector_sorted_copy[extract_eigenvector_sorted_copy['friend_of_rwth'] == True].loc[:426, ['friend_of_hsd', 'friend_of_rhein_waal', 'friend_of_niederrhein', 'friend_of_bochum', 'friend_of_koeln', 'friend_of_dortmund', 'friend_of_muenster', 'friend_of_rwth']].sum())
ahf.create_bar_plot_for_hs(hochschulen, values, 'Anzahl der Friends', 'Anzahl der Friends der Hochschulen bis Platz 425 der Eigenvector Centrality', 0.1)